#### Lecture 1. Object detection

1. Object detection 이란
- semantic segmentation : 객체(instance)를 구분하지 않음
- instance segmentation $\subset$ panoptic segmentation : 객체를 구분
- bounding box + classification 문제
- autonomous driving / OCR(optical caracter recognition)에 활용

2. two-stage detection
- gradient-based detector : 이미지의 특징에 대해 사람이 정교하게 engineering 후에 학습 가능한 부분은 단순한 SVM 이용
- selective search : over-segmentation (잘게 부숨), 비슷한 영역 통합 후 bounding box를 추출하여 물체의 후보군으로 삼음
- R-CNN(2014) : region 에 대해 warping 수행 후 영상인식에 사용된 fine tuning 된 CNN에 넣어 분류, SVM 으로 classifier 수행
    - hand-design된 selective search 사용(성능 한계), 작업 속도 느림
- fast R-CNN(2015) : feature 재활용으로 속도 향상 (약 18배)
    - conv feature map 추출 > ROI(region of interest) pooling
- faster R-CNN(2015) : region proposal 을 neural network로 대체(최초의 end-to-end)
    - IOU = $\frac{Area of Overlap}{Area of Union}$ 개념 : 두 영역의 정합도
    - Anchor box를 설정하고 Ground Truth와 IoU 가 0.7 이상 positive, 0.3이하 negative
    - region proposal network(RPN)에서 region proposal 제시된 것을 roi pooling 후 classification & bound-box regression 수행
    - k 개의 anchor box에 대해 2k 개의 object 판별 score(classification layer)와 4k 개의 coordinates(regression layer) 가 존재
    - Non-Maximum suppression(NMS) : bounding box filtering

3. single-stage detection
- 정확도를 조금 포기하고 real-time detection이 가능하도록
- Region proposal 과 선별적인 classification 없이 바로 detection
- YOLO(you only look once) 
    - input image를 grid로 쪼개고, 각 위치의 class probability map 추출 및 boundary regression 수행
    - conv 결과가 7x7(conv layer 해상도) x30 channel로 출력됨
        - (bounding box anchor 2 * {x,y,w,h,obj_score} 5) + class probaiblity(20)
- single shot multibox detector(SSD) : 정확성을 높이기 위해 prediction 횟수를 늘림
    - multi-scale object를 더 잘 처리할 수 있도록 중간 feature map을 각 해상도의 적절한 bounding box 들을 출력할 수 있도록 multi-scale 구조를 만듬
    - classifier 가 conv layer 각각의 출력층에서 작동함
    - total anchor box no = 8732로 많지만 단순한 network 구조로 빠른 속도
    - YoLO 뿐만 아니라 faster R-CNN 보다 좋은 성능
    
4. single stage vs two-stage
- single stage : ROI pooling 이 없으므로 모든 영역의 loss가 계산되고 gradient 발생
- 일반적인 영상은 배경이 더 많은 비중을 차지하므로 유용한 정보를 주지못하는 무수한 negative sample 이 존재(class imbalance)
- focal loss(cross-entropy 의 확장)
- RetinaNet(2017) : one-stage network, FPN + class/box prediction

5. Detection with Transformer
- DETR(facebook) : CNN image feature를 encoder input, object query를 decoder input

#### Lecture 2. CNN Visualization

1. CNN visualization
- CNN is blackbox
- CNN 시각화를 통해 성능이 잘나오는 원인과 개선점을 파악(debugging tool)
- ZFNet(2014) - 낮은 계층은 선의 방향, 형체에 대한 필터, middle/high는 고차원 형상
- filter visualization / activation visualization
- 뒤쪽 필터는 차원이 높아서 사람이 직관적으로 알아보기 어려움

2. model behavior analysis
- (model) parameter examination -- feature -- sensitivity -- decomposition (data)
- query image와 feature space 가 유사한 neighbor를 찾아보면 well clustered
    - pixel의 위치 정보가 아닌 feature를 잘 학습했음을 확인할 수 있음
- dimensionality reduction : 고차원 벡터를 눈으로 확인하기 쉬운 저차원 벡터로 변경
    - t-SNE : t-distributed stochastic neighbor embedding
- layer activation : behaviors of mid-to high level
    maximally activating patches - patch acquisition
- class visualizaiton - maximal class activation을 만드는 synthetic image를 생성
    - $ I^* =argmax_{I}f(I) - Reg(I)=argmax_{I}f(I) - \lambda \lVert I\rVert_2^2 $ (gradient ascent)
    
3. Model decision explanation
- saliency test(눈에 띄는 영역 검출) : 영상이 제대로 판정되기 위한 영역의 중요도 추출
    - occlusion map : mask 한 영역의 위치에 따라 prediction score가 달라짐
    - via backpropagation
    - rectified unit
    - guided backprop : backprop 과 Decov mask를 모두 사용함
- class activation mapping(CAM) : FC layer 대신 Global averae pooling(GAP) 이용
    - grad-CAM : backbone이 CNN이기만 하면 어떤 task에도 응용가능
    - SCOUTER : 영상 분류의 비교및 해석도 가능한 모델
- GAN dissection : 결과를 해석하는 모델에서 그치지 않고 응용도 가능함(생성 모델)

#### 실습 : Introduction to torch.autograd

Autograd : Automatic gradient calculation API
DL library (forward & backward) 를 구현하는 자동 미분
- propagation을 구성하는 chain rule 성분들을 모두 보관하고 있음

In [1]:
import torch

In [7]:
x = torch.randn(2, requires_grad=True) # x를 gradient를 저장할 수 있는 변수로 선언
y = x*3
gradients = torch.tensor([100, 0.1], dtype=torch.float)
y.backward(gradients, retain_graph = True) # graph를 보관하고 재사용 가능
print(x.grad)
y.backward(gradients)
print(x.grad) # gradients are accumulated

tensor([300.0000,   0.3000])
tensor([600.0000,   0.6000])


In [8]:
x = torch.randn(2, requires_grad = True)
y = x * 3
z = x / 2
w = x + y
w, y, z

(tensor([ 5.9066, -1.8393], grad_fn=<AddBackward0>),
 tensor([ 4.4300, -1.3795], grad_fn=<MulBackward0>),
 tensor([ 0.7383, -0.2299], grad_fn=<DivBackward0>))

Hooking : software components 사이의 함수를 가로채서 변화시키는 방법
register_hook : gradient를 낚아챌 수 있는 function

#### peer session

process: 코어수에 따라 병렬처리 가능(chrome window)
thread: 프로세스 위에 올라가있는 task(chrome tab)

- 쓰레드마다 갖는 메모리 공간 / 프로세스가 공유하는 메모리 공간이 있다.
- 프로세스가 늘어나면 쓰레드 공유 공간이 늘어나게 된다.
- 보통 1개의 process로 concurrent로 처리하는 것 보다, max core의 50~70%정도로 process를 나눠서 처리해주는 게 훨씬 좋은 성능을 낸다.
- python 은 GIL issue 떄문에 multi threading 으로 성능 향상이 어려움
- mini task) 으로 10만까지의 소수 찾고 성능 비교 후 github에 올리기 (~3.15 월)
- nested CV